## Predicting the Sale Price of Bulldozers (Kaggle Competition)

## 1. Problem Definition

The goal of this project is to predict the sale price of bulldozers at auction.  
Predictions are based on usage, equipment type, and configuaration.  
The data is sourced from auction result postings.  
Type of machine learning problem: **supervised learning / time series regression**

## 2. Evaluation

The competition evaluation metric was *root mean squared log error (RMSLE)*.  
**Project goal:** To minimize the difference between actual and predicted prices, i.e., to minimize RMSLE.

## 3. Data

Data is downloaded from the *Bluebook for Bulldozers* past Kaggle competition:  
[Bluebook for Bulldozers Kaggle Competition](https://www.kaggle.com/c/bluebook-for-bulldozers/overview)  
There are three main datasets:  
* **Train.csv** is the training set, which contains data through the end of 2011.
* **Valid.csv** is the validation set, which contains data from January 1, 2012 - April 30, 2012.  
The score on this set was used to create the public leaderboard.
* **Test.csv** is the test set, which contains data from May 1, 2012 - November 30, 2012.  
The score on this set determined the final rank for the competition.

## 4. Data Features

Kaggle provided a data dictiorany for these datasets.  
See `data-dictionary.xlsx` in the project folder.

#### Importing the tools

In [ ]:
### importing data analysis libraries
import numpy
from pandas import read_csv, DataFrame
from matplotlib import pyplot
from matplotlib.figure import Figure

### importing machine learning libraries
from sklearn.ensemble import RandomForestRegressor

#### Importing data

**Parsing dates**

Working with time series data requires date/time information to be in python datetime format for easy processing.  
Date/time columns are parsed into datetime format using the `parse_dates=` parameter of `read_csv()`.

In [ ]:
### importing bulldozer dataset from file
bulldozer_df: DataFrame = read_csv(
    filepath_or_buffer="../Large-Files/data-train-valid.csv",
    parse_dates=["saledate"],
    low_memory=False)

#### Verifying and preparing dataframe

In [ ]:
### displaying dataframe info
bulldozer_df.info()

In [ ]:
### checking for missing values
bulldozer_df.isna().sum()

**Sorting dataframe by date**

When working with time series, it is better to sort data by date/time.

In [ ]:
### sorting dataframe by sale date
bulldozer_sorted: DataFrame = bulldozer_df.sort_values(by="saledate", ascending=True)
bulldozer_sorted.head(20)

**Enriching dataframe with date information**

Using the `pandas.dt` attributes, all sorts of information can be extracted from datetime values.

In [ ]:
### creating new columns from sale date column (splitting sale date column)
bulldozer_sorted["saleYear"] = bulldozer_sorted["saledate"].dt.year
bulldozer_sorted["saleMonth"] = bulldozer_sorted["saledate"].dt.month
bulldozer_sorted["saleDay"] = bulldozer_sorted["saledate"].dt.day
bulldozer_sorted["saleDayOfWeek"] = bulldozer_sorted["saledate"].dt.day_of_week
bulldozer_sorted["saleDayOfYear"] = bulldozer_sorted["saledate"].dt.day_of_year

### deleting original sale date column
bulldozer_sorted = bulldozer_sorted.drop(columns="saledate")
bulldozer_sorted.head().T

**Feature engineering**

Feature engineering means processing data in the dataset.  
It includes transforming existing data and/or creating new data from existing data.

#### Exploring the target variable

In [ ]:
### plotting the distribution of target variable
bulldozer_sorted["SalePrice"].plot.hist(color="steelblue")

### customizing the plot
pyplot.title(label="Distribution of Bulldozer Sale Prices")
pyplot.ylabel(ylabel="Sale Count")
pyplot.xlabel(xlabel="Sale Price ($)");

#### Exploring feature: Sale date

In [ ]:
### plotting correlation between sale price and sale date
bulldozer_df[:500].plot.scatter(y="SalePrice", x="saledate", c="steelblue", s=50)

### customizing the plot
pyplot.title(label="Sale Price and Sale Date")
pyplot.ylabel(ylabel="Sale Price ($)")
pyplot.xlabel(xlabel="Sale Date");

#### Exploring feature: State of sale

In [ ]:
### plotting the distribution of sales by state
bulldozer_sorted["state"].value_counts().sort_index(ascending=True).plot.bar(figsize=(12,5))

### customizing the plot
pyplot.title(label="Distribution of Sales by States")
pyplot.ylabel(ylabel="Sale Counts");

## 5. Modeling

**Model-driven data exploration**

When there are a huge amount of features, it may be better to start building a machine learning model right away.  
Model-driven data exploration lets the machine learning algorithm select the most important features.

In [ ]:
### preparing a random forest regressor
regressor: RandomForestRegressor = RandomForestRegressor(random_state=42, n_jobs=-1)
regressor.fit(X=bulldozer_sorted.drop(columns="SalePrice"), y=bulldozer_sorted["SalePrice"])